## Setting Sun

Using backtracking to solve the [Setting Sun](https://en.wikipedia.org/wiki/Klotski) puzzle

In [20]:
import numpy as np

N_ROWS = 5
N_COLS = 4
VALID_COORDS = {(r, c) for r in range(N_ROWS) for c in range(N_COLS)}
WIN_COORDS = {(r, c) for r in range(3, 5) for c in range(1, 3)}
DIRECTIONS = ["LEFT", "RIGHT", "UP", "DOWN"]

class SettingSun:
    def __init__(self):
        self.board = np.zeros(shape=(N_ROWS, N_COLS))

    def setup(self):
        pieces = [1, 0, 0, 2, 1, 0, 0, 2, -1, 3, 3, -1, 4, 6, 7, 5, 4, 8, 9, 5]
        coords = [(r, c) for r in range(N_ROWS) for c in range(N_COLS)]

        for p, c in zip(pieces, coords):
            self.board[c] = p

    def get_empty_cells(self):
        """Returns coordinates of empty cells in board"""
        return self.get_piece_coords(-1)
    
    def get_sun(self):
        """Returns coordinates of the "sun" piece"""
        return set(self.get_piece_coords(0))
    
    def get_position(self):
        return str([self.get_piece_coords(p)[0] for p in range(10)])

    def get_piece_coords(self, piece):
        return [(r, c) for r, c in np.argwhere(self.board == piece)]
    
    def get_updated_piece_coords(self, piece, direction):
        match direction:
            case "LEFT":
                return {(r, c-1) for r, c in self.get_piece_coords(piece)}
            case "RIGHT":
                return {(r, c+1) for r, c in self.get_piece_coords(piece)}
            case "UP":
                return {(r-1, c) for r, c in self.get_piece_coords(piece)}
            case "DOWN":
                return {(r+1, c) for r, c in self.get_piece_coords(piece)}

    def update_board(self, piece, coords):
        """Updates the board, placing the piece specified to the coords specified.
        Called during "move" methods."""
        for r, c in coords:
            self.board[r, c] = piece

    def move_piece_left(self, piece):
        coords = self.get_updated_piece_coords(piece, "LEFT")
        self.board[np.where(self.board==piece)] = -1 # remove piece
        self.update_board(piece, coords)

    def move_piece_right(self, piece):
        coords = self.get_updated_piece_coords(piece, "RIGHT")
        self.board[np.where(self.board==piece)] = -1 # remove piece
        self.update_board(piece, coords)

    def move_piece_up(self, piece):
        coords = self.get_updated_piece_coords(piece, "UP")
        self.board[np.where(self.board==piece)] = -1 # remove piece
        self.update_board(piece, coords)

    def move_piece_down(self, piece):
        coords = self.get_updated_piece_coords(piece, "DOWN")
        self.board[np.where(self.board==piece)] = -1 # remove piece
        self.update_board(piece, coords)

    def is_valid_move(self, piece, direction):
        coords = self.get_updated_piece_coords(piece, direction)
        rows = {c[0] for c in coords}
        cols = {c[1] for c in coords}
        
        # Check that all coordinates are in-bounds.
        if not coords.issubset(VALID_COORDS) : return False

        # Check that there is space for the piece to move into.
        match direction:
            case "LEFT":
                l = {(r, min(cols)) for r in rows} # Coordinates to check if trying to move left.
                return {-1} == set([self.board[c] for c in l])
            case "RIGHT":
                r = {(r, max(cols)) for r in rows} # Coordinates to check if trying to move right.
                return {-1} == set([self.board[c] for c in r])
            case "UP":
                u = {(min(rows), c) for c in cols} # Coordinates to check if trying to move up
                return {-1} == set([self.board[c] for c in u])
            case "DOWN":
                d = {(max(rows), c) for c in cols} # Coordinates to check if trying to move down.
                return {-1} == set([self.board[c] for c in d])

        # Shouldn't be possible to make it here.
        raise Exception()

    def get_valid_moves(self):
        moves = {k:[] for k in range(10)}

        for piece in moves:
            for direction in DIRECTIONS:
                if self.is_valid_move(piece=piece, direction=direction):
                    moves[piece].append(direction)

        return moves

    def has_finished(self):
        return self.get_sun() == WIN_COORDS


In [21]:
s = SettingSun()
s.setup()
s.board

array([[ 1.,  0.,  0.,  2.],
       [ 1.,  0.,  0.,  2.],
       [-1.,  3.,  3., -1.],
       [ 4.,  6.,  7.,  5.],
       [ 4.,  8.,  9.,  5.]])

In [22]:
s.board[np.where(s.board == 0)] = 500
s.board[3, 1] = 0
s.board[4, 1] = 0
s.board[3, 2] = 0
s.board[4, 2] = 0

In [23]:
{(r, c) for r in range(3,5) for c in range(1,3)}.issubset(set(s.get_piece_coords(0)))

True

In [24]:
s.has_finished()

True

In [10]:
s.get_piece_coords(0)

[(3, 1), (3, 2), (4, 1), (4, 2)]

In [4]:
# Testing of is_valid_move method - Valid moves.
assert s.is_valid_move(piece=1, direction="DOWN")
assert s.is_valid_move(piece=2, direction="DOWN")
assert s.is_valid_move(piece=4, direction="UP")
assert s.is_valid_move(piece=5, direction="UP")
assert s.is_valid_move(piece=3, direction="LEFT")
assert s.is_valid_move(piece=3, direction="RIGHT")

In [5]:
s.get_position()

'[(0, 1), (0, 0), (0, 3), (2, 1), (3, 0), (3, 3), (3, 1), (3, 2), (4, 1), (4, 2)]'

In [ ]:
moves = {k:[] for k in range(10)}

for k in moves:
    for d in DIRECTIONS:
        if s.is_valid_move(piece=k, direction=d):
            moves[k].append(d)

moves

In [ ]:
coords = s.get_piece_coords(1)
rows = {x[0] for x in coords}
cols = {x[1] for x in coords}

In [ ]:
str([s.get_piece_coords(x)[0] for x in range(10)])

In [ ]:
l = {(x, min(cols) - 1) for x in rows} # Coordinates to check if trying to move left.
r = {(x, max(cols) + 1) for x in rows} # Coordinates to check if trying to move right.
u = {(min(rows) - 1, x) for x in cols} # Coordinates to check if trying to move up
d = {(max(rows) + 1, x) for x in cols} # Coordinates to check if trying to move down.

In [ ]:
{-1} == set([s.board[c] for c in d]) # Validation if piece can move